In [32]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# hyper parameters
LR = 0.01
n_embd = 100 # size of hidden layer
block_size = 20 # context length

%matplotlib inline
g = torch.Generator().manual_seed(2147483647) # for reproducibility

torch.__version__

'2.1.1+cu121'

In [33]:
import random
with open("assets/names.txt", "r+") as f:
	words = f.read().splitlines()
	words = [word.strip() for word in words] # get rid of any trailing spaces
	names = [w for w in words if w] # get rid of any empty strings
	
with open("assets/names.txt", "w") as f: 
	joined = "\n".join(names)
	f.write(joined)
min_chars = 1
max_chars = max(len(v) for v in names)
chars = sorted(list(set("".join(names))))

# in replacement of the start and end token. Every name should end with a period. and there should be no start token to begin a sequence
chars = ['.'] + chars
vocab_size = len(chars)
print("names: ", names[:5])
print("number of names: ", len(names))
print("(list of chars, count): ", ("".join(chars), vocab_size))
print("(max word length, min word length): ", (max_chars, min_chars))

atoi = {ch:i for i,ch in enumerate(chars)}
itoa = {i:ch for i,ch in enumerate(chars)}

# adding end token to each name
names = [list(name) + ['.'] for name in names]

names:  ['pascual', 'santana', 'paraskevi', 'shane', 'aidan']
number of names:  32033
(list of chars, count):  ('.abcdefghijklmnopqrstuvwxyz', 27)
(max word length, min word length):  (15, 1)


In [62]:
# model parameters
W1 = torch.randn(n_embd, vocab_size) # input to hidden
W2 = torch.randn(n_embd, n_embd) # hidden to hidden 
W3 = torch.randn(vocab_size, n_embd) # hidden to output
b1 = torch.zeros((n_embd, 1)) # bias for hidden layer
b2 = torch.zeros((vocab_size, 1)) # bias for output layer

# params = [W1, b1, W2, b2]
# for p in params:
# 	p.requires_grad = True

# structure of the model
# x -> embedding -> hidden layer -> tanh -> output -> softmax -> y


In [107]:
def loss(x, y, hprev):
	"""
	x -> input
	y -> target
	hprev -> hidden state from previous time step
	"""
	xs, hs, ys, ps = {}, {}, {}
	hs[-1] = hprev.copy()
	loss = 0
	for i in range(len(x)):
		xs[i] = torch.zeros((vocab_size, 1))
		x[i][x[t]] = 1
		hs[i] = torch.tanh(torch.dot(W1, xs[i]) + torch.dot(W2, hs[i-1]) + b1)
		ys[i] = F.softmax(torch.dot(W3, hs[i]) + b2, dim=0)

	
def sample(h, seed_ix, n):
	x = torch.zeros((vocab_size, 1))
	x[seed_ix] = 1
	ixes = []
	for i in range(n):
		l1 = torch.dot(W1, x) + torch.dot(W2, h) + b1
		l2 = F.tanh(l1)
		y = torch.dot(W3, l2) + b2
		p = F.softmax(y, dim=0)
		ix = torch.multinomial(p, num_samples=1, generator=g)
		x = torch.zeros((vocab_size, 0))
		x[ix] = 1
		ixes.append(ix.item())
	return ixes


In [105]:
word = "hello"
print("word: ", word)
ixes = F.one_hot(torch.tensor([atoi[ch] for ch in word]), vocab_size).float()


word:  hello


In [111]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('assets/names.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 228145 characters, 27 unique.
----
 hv
crwrmrkmzxpgcsoyqhaopdvmwmptvgrctffrtrzospfmzyvrwsdjhukttflqulqzyxzqqnzfxsspjizjbanjoftgtkzqp
tzizndwakopzgteezeywbvrzxslnozz
kxbcvtplsdqqzuswv
rurvrxkk
efmdhlueijyeobycilj
prpnmlerijxxsybpflhz
brr 
----
iter 0, loss: 82.395928
----
 
gf
nz
a
dsaiknacl
aua
aiknk
rianwua
koa
ammeg
vianbnacu
ahrnwirpa
alaiaelij
anagl
lsamfiqia
ldmwanayanfilvhekjluaskne
acayaye
tjkraianr
piaiqiang
lme
aiailnazmlaya
rvauailyanpezhaia
ara
a
aql
lesiaif 
----
iter 100, loss: 83.431205
----
 
eiaiahehrray
bhnnbntmt
neouieh
erbmmed
nskkbhkliklnaslnudoera
mh
aaitashel
ira
eunii
erisi
eau
jnluohe
jxinahlnbn
vi
ie
ebw
ai
qyosaebohr
fhgos
srakaj
quhioap
irkerny
nay
haln

kllrdaird
oeiilnbelabt 
----
iter 200, loss: 82.803108
----
 a
yml
anrramklan

arb
sha
ardrny
anrau
riaieyanmaya
ora
alzylmar
en
lmliah
hteamar
mk
daeet
andrnma
ame
argy
ljla

yzrlmame
ala
iae

lgynulclmi
eeonamcvorle
vkrakghanayeinjaalraginglrhuramaid
wvbrarah 
----
iter 300, loss: 81.915724


KeyboardInterrupt: 